# Total carbon per country

- In this code we calculate the total carbon pero country with a csv as an output with the values classified in different years
- summary of the process:
  - get both rasters and vector data
  - adapt a table for the output
  - iterate over the rasters and iterate over the vector data
  - do the masking and sum the values of the output raster (array)
  - collect the results into a list
  - append the results to the adapted final table

In [1]:
import os
import geopandas as gpd
import rasterio
import rasterio.mask
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
"""This is the location of the raster files"""
# ACHTUNG: the file name structure must be vcs_YYYY_global_300m.tif'
path = r"\\akif.internal\public\veg_c_storage_rawdata"

"""Get a list of the raster files inside the folder"""
File_list = [] #f for f in os.listdir(path) if os.isfile(mypath,f)
for file in os.listdir(path):
    if ".tif" in file:
        if file not in File_list:
            File_list.append(file)
    else:
        pass
print(File_list)

['vcs_2001_global_300m.tif', 'vcs_2002_global_300m.tif', 'vcs_2003_global_300m.tif', 'vcs_2004_global_300m.tif', 'vcs_2005_global_300m.tif', 'vcs_2009_global_300m.tif', 'vcs_2010_global_300m.tif', 'vcs_2011_global_300m.tif', 'vcs_2012_global_300m.tif', 'vcs_2013_global_300m.tif', 'vcs_2014_global_300m.tif', 'vcs_2015_global_300m.tif', 'vcs_2016_global_300m.tif', 'vcs_2017_global_300m.tif', 'vcs_2018_global_300m.tif', 'vcs_2019_global_300m.tif', 'vcs_2020_global_300m.tif']


In [3]:
"""Open the Vector data"""
globalmap = r"\\akif.internal\public\z_resources\im-wb\2015_gaul_dataset_mod_2015_gaul_dataset_gdba0000000b.shp"
gdf = gpd.read_file(globalmap)

"""Prepare the dataframe / df for the final table"""
df_final = pd.DataFrame(gdf.drop(columns='geometry'))
df_final = df_final.drop(["STATUS", "DISP_AREA", "ADM0_CODE", "STR0_YEAR", "EXP0_YEAR", "Shape_Leng", "ISO3166_1_", "ISO3166__1", "Shape_Le_1", "Shape_Area"], axis = 1)

In [ ]:
"""We set ourselves in the folder with the rasters"""
os.chdir(path)

"""Iterate the rasters"""
for file in File_list[:]: # CUIDATO CUANDO TERMINES BORRALO
    """Take the year of the file"""
    file_year = str(file[4:8])
    print("\r", "We are working with the file {} from the year {}".format(file, file_year), end="")

    """Create a list of the carbon values"""
    carbon_values = []

    """Open the raster"""
    with rasterio.open(file) as raster_file:

        """Iterate on the gdf"""
        for row_index, row in gdf.iterrows(): # gdf.loc[0:1].iterrows():
            geo_row = gpd.GeoSeries(row['geometry'])

            """Do the masking"""
            out_image, out_transform = rasterio.mask.mask(raster_file, geo_row, crop=True)

            """Sum the values ignoring the nan values"""
            carbon_total = np.nansum(out_image) # nansum treats nan values as 0, we have to do this since with sum we get as result nan

            """Append the value to the list"""
            carbon_values.append(carbon_total)

            print("\r", "the country {} is finished".format(row["ADM0_NAME"]), end="") #this is, so we delete the previous print
            
    print("\r", "Finished calculating {} year raster".format(file_year), end="")
        # time: for 1 year took 83m

    """Transform the list into a dataframe with the header of the year"""
    carbon_values_s = pd.DataFrame(carbon_values, columns = [file_year])

    """Append the dataframe to the final dataframe"""
    df_final = df_final.join(carbon_values_s)
    
"""Export the result"""
df_final.to_csv("total_carbon.csv")

Here is the same code separated in different blocks for testing

In [11]:
"""We set ourselves in the folder with the rasters"""
os.chdir(path)

"""Iterate the rasters"""
for file in File_list[6:7]:
    """Take the year of the file"""
    file_year = str(file[4:8])
    print("We are working with the file {} from the year {}".format(file, file_year))

We are working with the file vcs_2010_global_300m.tif from the year 2010


In [13]:
carbon_values = []

"""Iterate on the gdf"""
for row_index, row in gdf.loc[53:54].iterrows(): # gdf.loc[0:1].iterrows():
    geo_row = gpd.GeoSeries(row['geometry'])

    """Do the masking"""
    with rasterio.open(file) as raster_file:
        # band = raster_file.read(1)
        out_image, out_transform = rasterio.mask.mask(raster_file, geo_row, crop=True)

    """Sum the values ignoring the nan values"""
    carbon_total = np.nansum(out_image) # nansum treats nan values as 0, we have to do this since with sum we get as result nan

    """Append the value to the list"""
    carbon_values.append(carbon_total)

    print("the country {} is finished".format(row["ADM0_NAME"]))
    
print("Finished calculating {} Year".format(file_year))
# time: for 1 year took 83m

the country Cape Verde is finished
the country Côte d'Ivoire is finished
Finished calculating 2010 Year


In [34]:
"""Check if the result is correct"""
print(np.unique(out_image))
count_5 = np.count_nonzero(out_image == 5)
count_1 = np.count_nonzero(out_image == 1)
print(count_5, count_1)

[ 0.  1.  5. nan]
11422 7


In [50]:
"""Transform the list into a dataframe with the header of the year"""
carbon_values_s = pd.DataFrame(carbon_values, columns = [file_year])


In [51]:
"""Append the dataframe to the final dataframe"""
df_final_copy = df_final.join(carbon_values_s)

In [53]:
"""Export the result"""
df_final_copy.to_csv("total_carbon.csv")

In [ ]:
"""This is the output method for rasterio
the code expects 2 dimensions (height and width), so we 
have to transform the array from 3 to two dimensions
"""

"""Prepare the metadata for the output"""
out_band = raster_file.read(1)
out_meta = raster_file.meta
out_meta.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

"""testing write result"""
with rasterio.open(os.path.join(path, 'test.tif'), 'w', **out_meta) as dst:
    dst.write_band(1, out_image.astype(rasterio.float32))